<a href="https://colab.research.google.com/github/marinaceccon1/LearningFromNetworksProject/blob/main/Betweenness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computation with the second approach and the betweenness centrality

In [1]:
#import the libraries needed
import networkx as nx
import numpy as np
import sys
import collections
import random

In [2]:
#clone the github repository containing the dataset
!git clone https://github.com/marinaceccon1/LearningFromNetworksProject.git

Cloning into 'LearningFromNetworksProject'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 36 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (36/36), 6.20 MiB | 2.45 MiB/s, done.


In [3]:
# load the data and create the graph representing the network
matrix = np.loadtxt('/content/LearningFromNetworksProject/out.txt', usecols=range(2))
G = nx.from_edgelist(matrix)

In [4]:
# computes the local clustering coefficient and order the dictionaries w.r.t. the keys
lcc = nx.clustering(G)
ordered_lcc = collections.OrderedDict(sorted(lcc.items()))

Read the betweenness centrality values from the bet.txt file

In [5]:
# Open the file with 'read' mode
f = open('/content/LearningFromNetworksProject/bet.txt', 'r')

# Read all lines in the file
lines = f.readlines()

# Close the file
f.close()

# Create an empty list
bet_centralities = []

# Iterate over the lines and append the values to the list
for line in lines:
  bet_centralities.append(float(line))

In [6]:
#create a dictionary from the list bet_centralities
# the keys of the dictionary are the node indices, the values is the betweenness centrality associated to that node
bet_centr = {}
for i in range(len(bet_centralities)):
  bet_centr[i+1] = bet_centralities[i]

In [7]:
#sort the dictionary in increasing order w.r.t. the values (reserve = False)
sorted_lcc = sorted(ordered_lcc.items(), key=lambda x:x[1], reverse = False)

In [8]:
#save in len_sorted_lcc the number of values below the threshold
for i in range(len(sorted_lcc)):
  if sorted_lcc[i][1] > 0.24:
    len_sorted_lcc = i
    break

In [9]:
#sort the dictionary in decreasing order w.r.t. the values(reverse = True)
sorted_bet_centr = sorted(bet_centr.items(),  key=lambda x:x[1], reverse = True)

In [10]:
#save in len_sorted_bet_centr the number of values above the threshold
for i in range(len(sorted_bet_centr)):
  if sorted_bet_centr[i][1] < 0.95:
    len_sorted_bet_centr = i
    break

In [11]:
#creates a list with the first len_sorted_lcc keys in the sorted_lcc list
new_sorted_lcc = []
for i in range(len_sorted_lcc):
  new_sorted_lcc.append(sorted_lcc[i][0])
s1 = set(new_sorted_lcc) #converts the list into a set, s1 eventually contains the indices of the nodes below the threshold

In [12]:
# creates a list wit the first len_sorted_bet_centr keys in new_sorted_bet_centr
new_sorted_bet_centr = []
for i in range(len_sorted_bet_centr):
  new_sorted_bet_centr.append(sorted_bet_centr[i][0])
s2 = set(new_sorted_bet_centr) #converts the list into a set, s2 eventually contains the indices of the nodes above the threshold

In [13]:
# computes the intersection
s = s1 & s2

In [14]:
node_list = list(s)

In [15]:
cardinalities = {} #new dictionary, the keys will be pairs of nodes that are not adjacent, the value is the cardinality 
#of the intersection of their neighbourhoods

for i in range(len(node_list)): #we consider all the nodes in this list, i.e. all the nodes with associated metrics within the thresholds
  for j in range(i+1,len(node_list)): #in order not to contemplate the same pair twice, first as (i,j) and after as (j,i), we set j>i
    
    breakout_flag = False #needed to compute a double break
    for key,value in (G.adj[node_list[i]]).items(): #check the neighbours of node_list[i]     
      if key == node_list[j]: #if node_list[j] is a neighbour of node_list[i]
        breakout_flag = True
        break #exit the for key,value loop
    if breakout_flag:
      break #exit the for j loop

    s1 = set(G.adj[node_list[i]]) #convert the dictionary in a set containing all the keys in the dictionary
    s2 = set(G.adj[node_list[j]])
    res = s1 & s2 #keys that belong to both sets

    cardinalities[(node_list[i], node_list[j])] = len(res) #the value associated to that pair 
    #of nodes is the cardinality of the intersection computed above

inverse_sorted_card = sorted(cardinalities.items(), key=lambda x:x[1], reverse = True) #order the dictionary by value (decreasing order)

In [16]:
N = 500

In [ ]:
#add the firt N edges of inverse_sorted_card to a copy of G
from builtins import sum as siumm

G_1 = G.copy()
for i in range(N):
  u = inverse_sorted_card[i][0][0]
  v = inverse_sorted_card[i][0][1]
  G_1.add_edge(u,v)

glo_clu_coeff = nx.transitivity(G_1) #compute the global clustering coefficient of the new graph
triangles = siumm(nx.triangles(G_1).values())/3 #compute the number of triangles of the new graph
print(glo_clu_coeff) 
print(triangles)